# Definition of a cavity mode and light interacting with it.


Driving from two field directions at once. Now we completely the leave the SVEA in terms of propagation direction. We model every wavelength. The forward going wave propagates as $e^{ikz}$ and the backward going wave propagates as $e^{-ikz}$. This would mean we'd have to model $>10^4$ waveleneghts for typical media sizes. However, the result is generally the same for thin media as thick, so long as the OD is similar. That is, we can usually get away with modelling a much thinner medium (say 20 wavelengths long) and scaling up the atom re-radiation strength to give the same OD as the large medium.

The main reason to go to this effort is (I think) to look at the effects of a cavity. To do this is actually just a step further.

Ultimately the driving field for the atoms will be given by a state dependent value $E_{tot}$ which has contributions from $E_{inp,F}(t), E_{inp,R}$ and $E_{pol}(t, \rho)$. Each step it's calculating the space dependent polarisation $E_{pol}(z)$, then using this to calculate the forward and backwards moving polarisation due to atoms alone. Then use the final value to calculate the contribution of any extra modes (e.g due to reflections due to a cavity) who's spatial dependence is pre-calcuated. These weighted modes are then summed with the moving polarisation modes to give $E_{tot}(z)$

### Modes and scattering of a Fabry-Perot Cavity 

This is poorly explained, _but_:

We need a way to take the current values of input fields (fw and rv) as well as fields generated by polarisation (forward and reverse) and gives us the degree of forward and backward moving waves in the cavity. 

The generated 'scattering' matrix is used only in the calc_total_field function below.


In [1]:
def get_cavity_matrix(R1_val, R2_val, kL_val):
    """ Calculate a scattering matrix for the given cavity parameters. See seperate notebook (TODO!)
    Also, this includes no phase change on reflection inside the cavity (which may well be dodgy)
    """
    r1, r2, t1, t2= sm.symbols("r_1, r_2, tau_1, tau_2") # These are mirror transmisitivies/reflectivities 
                    #in terms of field. Could be complex, in principle to represent phase change (but won't be here)
    kL = sm.symbols("kL") # Just wave-vector x L (i.e. number of wavelengths between the mirrors)
    eps, A= sm.symbols('epsilon, A')
    
    subsD={A: 1/(1-r1*r2*sm.exp(2*sm.I*kL)), 
           eps: sm.exp(sm.I*kL), 
           t1:sm.sqrt(1-r1**2 ), t2:sm.sqrt(1-r2**2) }
    MS = sm.Matrix([
            [t1, t2*eps*r1, r2*eps*r1, r1], # x [EinFw, EinRv, EpolBack, EpolFront] = E_modefw 
            [t1*eps*r2, t2, r2, r1*eps*r2],  # x [EinFw, EinRv, EpolBack, EpolFront] = E_moderv
            [t1*eps*t2, t2*eps*r1*eps*t2 - r2, t2, r1*eps*t2], # x [] = E_out_fw
            [t1*eps*r2*eps*t1 - r1, t2*eps*t1, r2*eps*t1, t1] # x [] = E_out_rv

        ])*A
    
    M = MS.subs(subsD).subs({kL: kL_val, r1:np.sqrt(R1_val), r2:np.sqrt(R2_val) })
    M = np.array(M, dtype='c16') # More convenient as a 2d numpy array.


    return M

